In [18]:
import pymongo

print(pymongo.__version__)

conn = pymongo.MongoClient(
    host="192.168.0.6",
    port=29019,
    username="amy",
    password="zhd!!@4557",
)
str_database_name = "tags_test_db"
db = conn.get_database(str_database_name)

str_collection_name = "location_tags_table_v2"
# db.drop_collection(str_collection_name)
collection = db.get_collection(str_collection_name)

3.12.0


In [19]:
import pandas as pd
df = pd.read_csv('./boolocally_poi.csv/선별 테마와 비교-표 1.csv')
df.head()

,POI,커스텀맵 주제,커스텀맵 주제.1,커스텀맵 주제.2,운영팀 테마3,태그 추출 태그 top3,태그 추출 상세,태그 관련 하위어 점수
0,1723,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
1,외할머니1723솜씨,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
2,목련을부탁해,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
3,명천재,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN
4,차경,"고요한 쉼의 공간, 소박한 마당이 있는 전주 한옥 카페",한옥,음식점,카페,NaN,NaN,NaN


In [20]:
poi_names = df['POI'].tolist()
poi_names[:2]

['1723', '외할머니1723솜씨']

In [30]:
for poi_name in poi_names[:1]:
    print(poi_name)
    results = collection.find({'sns_name': {'$regex': poi_name}})
    for result in results[:1]:
        print(result)

1723
{'_id': 2206530946287836, 'sns_name': 'Cafe 1723', 'num_of_documents': 116, 'raw_keywords_score': [['전주', 12.885863834166516], ['카페', 6.521926852562766], ['빈티지', 4.058319376793317], ['디저트', 3.867649923756334], ['맛스타그램', 3.2730393230959143], ['데일리', 3.0976056027014836], ['일상', 2.867812905210562], ['맞팔', 2.7679246369215695], ['셀스타그램', 2.755781276601813], ['사진', 1.9499517737251586], ['먹스타그램', 1.7869919135774817], ['좋아요', 1.7434491840775328], ['뱅쇼', 1.7392448539999132], ['컵홀더', 1.637765658289002], ['한옥마을', 1.593663357708781], ['커피', 1.510115487286606], ['인절미', 1.4382263395310226], ['아메리카노', 1.4078594790115946], ['아이돌', 1.405698064307317], ['까페', 1.313365881048373], ['풍경', 1.223737273080515], ['비투비', 1.1861471392167635], ['얼스타그램', 1.0415982659755483], ['셀피', 0.9262987567260769], ['마마무', 0.9240607654676626], ['일칠이삼', 0.7102321430642761], ['소통', 0.6996837260707651], ['로스팅', 0.6859263931380492], ['이벤트', 0.6705485776271943], ['기념일', 0.6569155349610284], ['여행', 0.6260688153528213], ['해외여', 